<a href="https://colab.research.google.com/github/abdou-brahmia/Face-Recognition-in-Colab-Notebook/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Face Recognition in Colab Notebook

**Set Up Environment**

In [1]:
%%capture
!pip install face_recognition

## 1. Import Necessary Libraries

In [2]:
import  matplotlib.pyplot as plt
from google.colab import files
from numpy import savetxt
import face_recognition
import requests
import pickle
import json
import cv2
import os

## 2. Upload Known and Unknown Faces


In [3]:
# Define folder names
folder_names = ["known_faces", "unknown_faces"]

# Loop through each folder
for folder_name in folder_names:
    # Create the folder if it doesn't exist
    folder_path = os.path.join("/content", folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # Upload files to the folder
    uploaded_files = files.upload()
    for filename, content in uploaded_files.items():
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'wb') as file:
            file.write(content)

    # Print the list of uploaded files in the folder
    print(f"Files uploaded to folder '{folder_name}': {os.listdir(folder_path)}")

Saving biden.jpg to biden.jpg
Saving Cristiano Ronaldo.jpg to Cristiano Ronaldo.jpg
Saving Houari Boumediene.jpg to Houari Boumediene.jpg
Saving obama.jpg to obama.jpg
Files uploaded to folder 'known_faces': ['biden.jpg', 'Cristiano Ronaldo.jpg', 'obama.jpg', 'Houari Boumediene.jpg']


Saving 6000.jpg to 6000.jpg
Saving biden3.jpg to biden3.jpg
Saving bou2.jpg to bou2.jpg
Saving bou4.jpg to bou4.jpg
Saving ob2.jpg to ob2.jpg
Saving ob3.jpg to ob3.jpg
Files uploaded to folder 'unknown_faces': ['biden3.jpg', 'ob2.jpg', 'bou2.jpg', 'ob3.jpg', 'bou4.jpg', '6000.jpg']


## 3. Face Recognition Functions

**Load Known Faces for Training**

In [4]:
def load_known_faces(known_faces_folder):
    known_faces = []
    known_face_names = []

    for filename in os.listdir(known_faces_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(known_faces_folder, filename)
            image = face_recognition.load_image_file(image_path)
            face_encoding = face_recognition.face_encodings(image)[0]
            known_faces.append(face_encoding)
            known_face_names.append(os.path.splitext(filename)[0])

    return known_faces, known_face_names

**Recognize Faces in Unknown Images**

In [5]:
def recognize_faces(unknown_faces_folder, known_faces, known_face_names):
    for filename in os.listdir(unknown_faces_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(unknown_faces_folder, filename)
            unknown_image = face_recognition.load_image_file(image_path)
            unknown_face_locations = face_recognition.face_locations(unknown_image)
            unknown_face_encodings = face_recognition.face_encodings(unknown_image, unknown_face_locations)

            for (top, right, bottom, left), unknown_face_encoding in zip(unknown_face_locations, unknown_face_encodings):
                matches = face_recognition.compare_faces(known_faces, unknown_face_encoding)
                name = "Unknown"

                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]

                print(f"{name} found in {filename}")

## 4. Save and Load Known Faces Data


```
  Save Known Faces Data to File
  Load Known Faces Data from File
```



In [6]:
def save_known_faces(known_faces, known_face_names, file_path):
    with open(file_path, 'wb') as file:
        data = {'known_faces': known_faces, 'known_face_names': known_face_names}
        pickle.dump(data, file)

def load_known_faces_from_file(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        return data['known_faces'], data['known_face_names']

## 5. Main Function

In [7]:
known_faces_folder = "/content/known_faces"
unknown_faces_folder = "/content/unknown_faces"

In [8]:
def main(known_faces_folder):
    known_faces, known_face_names = load_known_faces(known_faces_folder)

    # Save known faces data
    save_known_faces(known_faces, known_face_names, 'known_faces_data.pkl')

    # For camera test
    video_capture = cv2.VideoCapture(0)

    # Load known faces data from file
    # If you want to use the previously saved known faces data, uncomment the following line:
    # loaded_known_faces, loaded_known_face_names = load_known_faces_from_file('known_faces_data.pkl')

    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        # Recognize faces and display the result
        frame_with_recognition = recognize_faces(frame, known_faces, known_face_names)

        # Display the resulting frame
        plt.imshow(frame_with_recognition[:, :, ::-1])  # Convert BGR to RGB for Matplotlib
        plt.show()

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close all OpenCV windows
    video_capture.release()
    cv2.destroyAllWindows()


## 6. Execute the Code

**Execute face recognition on known and unknown faces**

In [9]:
if __name__ == "__main__":
    known_faces, known_face_names = load_known_faces(known_faces_folder)
    recognize_faces(unknown_faces_folder, known_faces, known_face_names)

biden found in biden3.jpg
obama found in ob2.jpg
Houari Boumediene found in bou2.jpg
obama found in ob3.jpg
Houari Boumediene found in bou4.jpg
biden found in 6000.jpg


**Execute main function for camera testing**

In [ ]:
if __name__ == "__main__":
    main(known_faces_folder)